Weil für die Analyse bez. der Abhängigkeit von (linken) Mehrheiten Kooperation über den linken Block (SP,GP,AL) womöglich interessanter sind, konstruieren wir hier doch einen entsprechenden Dataframe

In [63]:
#Hier definieren wir die drei Blöcke
Block_Links = ["SP", "Grüne", "AL", "FraP", "CSP"]
Block_Mitte = ["CVP", "GLP", "EVP", "LdU", "Parteilos"]
Block_Rechts = ["FDP", "SVP", "PFZ", "SL"]

In [64]:
import os
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re

files = os.listdir("alle_geschaefte")
df_list = []

for file in tqdm(files): 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("alle_geschaefte/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        #Jetzt filtern wir nur jene raus, die mehrere Einreichende haben:
        einreichende = soup.find_all("div")[24].find_all("a")
        if len(einreichende) > 1: 
            
            #Jetzt filtern wir jene, die überhaupt einen Namen drin haben; Erkennungsmerkmal: Klammern
            einreichender1 = einreichende[0].text.strip()
            if "(" and ")" in einreichender1:
                
                name1 = einreichender1.split(" (")[0]
                partei1 = einreichender1.split(" (")[1][:-1]
                
                einreichender2 = einreichende[1].text.strip()
                name2 = einreichender2.split(" (")[0]
                partei2 = einreichender2.split(" (")[1][:-1]
                
                geschaeft_nr = soup.find_all("div")[22].find_all("span")[1].text
                
                regex = r"((?<=\svom\s)\d*\.\d*\.\s*\d*)"
                datum = re.findall(regex, str(soup))[0]
                
                try: 
                    regex_dep = r"((?<=Zuständiges Departement).*\n*.*)"
                    dep_fragment = re.findall(regex_dep, str(soup))[0]
                    soup_dep = BeautifulSoup(dep_fragment, "html.parser")
                    departement = soup_dep.text[1:]

                except: 
                    departement = ""
                            
                #Zum Schluss: Wir wollen nur jene in unserem Dict mit unterschiedlichen Blocks
                if partei1 in Block_Links:
                    block1 = "links"
                elif partei1 in Block_Mitte:
                    block1 = "mitte"
                else:
                    block1 = "rechts"
                    
                if partei2 in Block_Links:
                    block2 = "links"
                elif partei2 in Block_Mitte:
                    block2 = "mitte"
                else:
                    block2 = "rechts"
                
                if block1 != block2:
                    mini_dict = {
                        "Datum" : datum,
                        "Geschäft" : geschaeft_nr,
                        "Einreichender 1" : name1,
                        "Partei 1" : partei1,
                        "Block 1" : block1,
                        "Einreichender 2" : name2,
                        "Partei 2" : partei2,
                        "Block 2" : block2,
                        "Departement" : departement
                    }
                    df_list.append(mini_dict)  

100%|██████████| 7026/7026 [04:42<00:00, 24.84it/s]


In [89]:
#Jetzt ist die Liste nicht mehr so lang...
len(df_list)

541

In [90]:
df = pd.DataFrame(df_list)
df.head(5)

,Datum,Geschäft,Einreichender 1,Partei 1,Block 1,Einreichender 2,Partei 2,Block 2,Departement
0,25.11.2009,2009/562,Kurt Hüssy,SVP,rechts,Peter Anderegg,EVP,mitte,Polizeidepartement (PD)
1,28.09.2011,2011/362,Alecs Recher,AL,links,Maleica Landolt,GLP,mitte,Finanzdepartement (FD)
2,20.10.2004,2004/541,Niklaus Scherr,AL,links,Kurt Maeder,CVP,mitte,Hochbaudepartement (HBD)
3,16.05.2013,2005/550,Mario Mariani,CVP,mitte,Muriel Herzig,Grüne,links,Tiefbau- und Entsorgungsdepartement (TED)
4,06.01.2016,2016/16,Christina Schiller,AL,links,Mario Babini,Parteilos,mitte,Polizeidepartement (PD)


In [91]:
#Hats hier noch zusätzliche Leute ohne Parteiangabe, so dass wir das Reparatur-Dict ergänzen müssen? Nein!
df[df["Partei 2"] == ""]["Einreichender 2"].value_counts()

Anton Stäbler          8
Gregor Bucher          5
Placidus Maissen       4
Urs Rechsteiner        4
Andreas Ammann         4
Esther Ponti-Weder     2
Fakir Atalay           1
Urs Lauffer            1
Andreas J. Schlegel    1
Name: Einreichender 2, dtype: int64

In [92]:
Reparatur_dict = {"Gregor Bucher":"Grüne",
                  "Andreas Ammann":"SP",
                  "Urs Rechsteiner":"CVP",
                  "Andreas J. Schlegel":"FDP",
                  "Jürg Liebermann":"FDP",
                  "Peter Marti":"FDP",
                  "Fakir Atalay":"SP",
                  "Anton Stäbler":"CVP",
                  "Placidus Maissen":"CVP",
                  "Salomon Browar":"Grüne",
                  "Monika Piesbergen":"FDP",
                  "Hanna Lienhard":"FDP",
                  "Urs Lauffer":"FDP",
                  "Esther Ponti-Weder":"CVP"}

In [93]:
def Reparatur(Person):
    return Reparatur_dict[Person]

fehlerposition1 = df[df["Partei 1"] == ""]["Einreichender 1"].index.tolist()
fehlerposition2 = df[df["Partei 2"] == ""]["Einreichender 2"].index.tolist()

for zeile in fehlerposition1:
    df.loc[zeile, 'Partei 1'] = df.loc[zeile, 'Partei 1'].replace("", Reparatur(df.loc[zeile, "Einreichender 1"]))

for zeile in fehlerposition2:
    df.loc[zeile, 'Partei 2'] = df.loc[zeile, 'Partei 2'].replace("", Reparatur(df.loc[zeile, "Einreichender 2"]))

Achtung, durch die Reparatur sind möglicherweise Paare der gleichen Partei oder des gleichen Blocks entstanden, die müssen wir rausfiltern:

In [94]:
len(df)

541

In [83]:
df.loc[0, "Partei 2"]

'EVP'

In [84]:
#Hier ist die Variante für Parteien:
for zeile in range(0, len(df)-1):
    if df.loc[zeile, "Partei 1"] == df.loc[zeile, "Partei 2"]:
        df.drop(zeile, inplace = True)

In [85]:
len(df)

493

In [95]:
#Und hier ist die Variante für Blöcke:
#Hier definieren wir die drei Blöcke
Block_Links = ["SP", "Grüne", "AL", "FraP", "CSP"]
Block_Mitte = ["CVP", "GLP", "EVP", "LdU", "Parteilos"]
Block_Rechts = ["FDP", "SVP", "PFZ", "SL"]

for zeile in range(0, len(df)-1):
    
    #erst müssen wir die fehlende Wert für die Blöcke ergänzen...
    if df.loc[zeile, "Partei 1"] in Block_Links:
        df.loc[zeile, "Block 1"] = "links"
    elif df.loc[zeile, "Partei 1"] in Block_Mitte:
        df.loc[zeile, "Block 1"] = "mitte"
    else:
        df.loc[zeile, "Block 1"] = "rechts"
                    
    if df.loc[zeile, "Partei 2"] in Block_Links:
        df.loc[zeile, "Block 2"] = "links"
    elif df.loc[zeile, "Partei 2"] in Block_Mitte:
        df.loc[zeile, "Block 2"] = "mitte"
    else:
        df.loc[zeile, "Block 2"] = "rechts"
    
    #...und dann löschen wir gleiche raus
    if df.loc[zeile, "Block 1"] == df.loc[zeile, "Block 2"]:
        #print( df.loc[zeile, "Block 1"] + df.loc[zeile, "Block 2"])
        df.drop(zeile, inplace = True)

In [96]:
len(df)

478

In [97]:
#Auch diesen Dataframe speichern wir:
df.to_csv("Dataframe_bereinigt_blockuebergreifend.csv")

### Die Zahlen können wir schon einmal auswerten:

In [59]:
df_Anzahl = df["Partei 1"].value_counts().to_frame()
df_Anzahl["Partei 2"] = df["Partei 2"].value_counts()
df_Anzahl["Geschaefte"] = df_Anzahl["Partei 1"] + df_Anzahl["Partei 2"]
df_Anzahl = df_Anzahl.drop(columns=['Partei 1', 'Partei 2'])
df_Anzahl

,Geschaefte
SP,196.0
Grüne,96.0
FDP,133.0
GLP,104.0
CVP,139.0
AL,31.0
SVP,43.0
Parteilos,16.0
EVP,42.0
PFZ,NaN


In [60]:
df[df["Einreichender 1"] == "Albert Leiser"]

,Datum,Geschäft,Einreichender 1,Partei 1,Block 1,Einreichender 2,Partei 2,Block 2,Departement
20,24.3.2004,2004/150,Albert Leiser,FDP,rechts,Peter Püntener,GLP,mitte,Schul- und Sportdepartement (SSD)
307,13.6.2007,2007/348,Albert Leiser,FDP,rechts,Niklaus Scherr,AL,links,Tiefbau- und Entsorgungsdepartement (TED)
372,30.10.2002,2002/446,Albert Leiser,FDP,rechts,Robert Schönbächler,CVP,mitte,Präsidialdepartement (PRD)
428,04.09.2019,2017/263,Albert Leiser,FDP,rechts,Andreas Kirstein,AL,links,Tiefbau- und Entsorgungsdepartement (TED)
433,19.08.2009,2009/355,Albert Leiser,FDP,rechts,Daniel Leupi,Grüne,links,Tiefbau- und Entsorgungsdepartement (TED)


In [61]:
df[df["Einreichender 2"] == "Albert Leiser"].sort_values("Datum", ascending = False)

,Datum,Geschäft,Einreichender 1,Partei 1,Block 1,Einreichender 2,Partei 2,Block 2,Departement
318,9.6.2004,2004/300,Niklaus Scherr,AL,links,Albert Leiser,FDP,rechts,Tiefbau- und Entsorgungsdepartement (TED)
340,31.3.2004,2004/164,Gerold Lauber,CVP,mitte,Albert Leiser,FDP,rechts,Sozialdepartement (SD)
14,31.01.2007,2007/52,Daniel Meier,CVP,mitte,Albert Leiser,FDP,rechts,Hochbaudepartement (HBD)
310,19.06.2013,2013/234,Niklaus Scherr,AL,links,Albert Leiser,FDP,rechts,Tiefbau- und Entsorgungsdepartement (TED)
353,12.04.2017,2017/105,Andreas Kirstein,AL,links,Albert Leiser,FDP,rechts,Tiefbau- und Entsorgungsdepartement (TED)
254,12.03.2008,2008/120,Mario Mariani,CVP,mitte,Albert Leiser,FDP,rechts,Hochbaudepartement (HBD)
3,1.12.2004,2004/637,Corine Mauch,SP,links,Albert Leiser,FDP,rechts,Finanzdepartement (FD)
211,08.01.20,2006/387,Niklaus Scherr,AL,links,Albert Leiser,FDP,rechts,Tiefbau- und Entsorgungsdepartement (TED)
422,07.11.2018,2018/423,Marco Denoth,SP,links,Albert Leiser,FDP,rechts,Hochbaudepartement (HBD)
8,06.03.2019,2019/84,Ernst Danner,EVP,mitte,Albert Leiser,FDP,rechts,Tiefbau- und Entsorgungsdepartement (TED)


In [62]:
df["Departement"].value_counts()

Tiefbau- und Entsorgungsdepartement (TED)                      113
Hochbaudepartement (HBD)                                        47
Polizeidepartement (PD)                                         37
Schul- und Sportdepartement (SSD)                               36
Präsidialdepartement (PRD)                                      34
Departement der Industriellen Betriebe (DIB)                    33
Sozialdepartement (SD)                                          31
Finanzdepartement (FD)                                          28
Gesundheits- und Umweltdepartement (GUD)                        20
Sicherheitsdepartement (SID)                                    19
Hochbaudepartement (HBD), Schul- und Sportdepartement (SSD)      2
                                                                 2
Name: Departement, dtype: int64

### Jetzt zählen wir nach Legislaturen

bei diesen Positionen stimmt beim Datum etwas Grundsätzliches nicht, weshalb wir es mit einem annäherungsweisen Datum  aufgrund der Geschäftnummer ersetzen:

In [98]:
fehlerliste = ["29.01.20","08.01.20","18.12.19"]

for fehler in fehlerliste:
    fehlerposition = df[df["Datum"] == fehler].index.tolist()
    for zeile in fehlerposition:
        Jahr = df.loc[zeile, "Geschäft"].split("/")[0]
        Ersatzdatum = "31.12." + Jahr
        df.loc[zeile, "Datum"] = df.loc[zeile, "Datum"].replace(fehler, Ersatzdatum)

In [99]:
df["Datum"] = pd.to_datetime(df['Datum'], format='%d.%m.%Y')
df = df.set_index("Datum")
df.head(5)

,Geschäft,Einreichender 1,Partei 1,Block 1,Einreichender 2,Partei 2,Block 2,Departement
Datum,,,,,,,,
2009-11-25,2009/562,Kurt Hüssy,SVP,rechts,Peter Anderegg,EVP,mitte,Polizeidepartement (PD)
2011-09-28,2011/362,Alecs Recher,AL,links,Maleica Landolt,GLP,mitte,Finanzdepartement (FD)
2004-10-20,2004/541,Niklaus Scherr,AL,links,Kurt Maeder,CVP,mitte,Hochbaudepartement (HBD)
2013-05-16,2005/550,Mario Mariani,CVP,mitte,Muriel Herzig,Grüne,links,Tiefbau- und Entsorgungsdepartement (TED)
2016-01-06,2016/16,Christina Schiller,AL,links,Mario Babini,Parteilos,mitte,Polizeidepartement (PD)


In [100]:
df_1990 = df["1990-05-15":"1994-05-14"]
df_1994 = df["1994-05-15":"1998-05-14"]
df_1998 = df["1998-05-15":"2002-05-14"]
df_2002 = df["2002-05-15":"2006-05-14"]
df_2006 = df["2006-05-15":"2010-05-14"]
df_2010 = df["2010-05-15":"2014-05-14"]
df_2014 = df["2014-05-15":"2018-05-14"]
df_2018 = df["2018-05-15":"2022-05-14"]

print("1990-94: "+str(len(df_1990)))
print("1994-98: "+str(len(df_1994)))
print("1998-02: "+str(len(df_1998)))
print("2002-06: "+str(len(df_2002)))
print("2006-10: "+str(len(df_2006)))
print("2010-14: "+str(len(df_2010)))
print("2014-18: "+str(len(df_2014)))
print("2018-22: "+str(len(df_2018)))

1990-94: 0
1994-98: 7
1998-02: 39
2002-06: 79
2006-10: 93
2010-14: 91
2014-18: 102
2018-22: 67
